Basic notebook for applying a simple sklearn prediction method on the kaggle titanic dataset

In [76]:
import os

In [77]:
DATASET_COMP = 'titanic'
DATA_DIR = './data'
DATA_ZIP = 'titanic.zip'
TRAIN_CSV = 'train.csv'
TEST_CSV = 'test.csv'

VALIDATION_SPLIT = 0.2
TARGET_VAR = 'Survived'

In [78]:
# load and prep datset from kaggle
if not os.listdir(DATA_DIR):
    print('data folder is empty. Downloading data...')
    from kaggle.api.kaggle_api_extended import KaggleApi
    api = KaggleApi()
    api.authenticate()

    api.competition_download_files(DATASET_COMP, DATA_DIR, quiet = False)

if not os.path.exists(os.path.join(DATA_DIR,TRAIN_CSV)) and os.path.exists(os.path.join(DATA_DIR,DATA_ZIP)):
    print('data set downloaded, unzipping...')
    import zipfile
    with zipfile.ZipFile(os.path.join(DATA_DIR, DATA_ZIP), 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR)


In [79]:
# load data into pandas dataframe
import pandas as pd
from sklearn.model_selection import train_test_split

train_df = pd.read_csv(os.path.join(DATA_DIR, TRAIN_CSV))
test_df = pd.read_csv(os.path.join(DATA_DIR, TEST_CSV))

# create train and validation set from train_df, splitting out target column
X_train, X_val, y_train, y_val = train_test_split(train_df.drop(TARGET_VAR, axis = 1), train_df[TARGET_VAR], test_size = VALIDATION_SPLIT, random_state = 42)

In [80]:
# take a quick look at the tarining data set
print('X_train shape: ', X_train.shape)
print('X_val shape: ', X_val.shape)
print('y_train shape: ', y_train.shape)
print('y_val shape: ', y_val.shape)

X_train.head()


X_train shape:  (712, 11)
X_val shape:  (179, 11)
y_train shape:  (712,)
y_val shape:  (179,)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
331,332,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,C124,S
733,734,2,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,NaN,S
382,383,3,"Tikkanen, Mr. Juho",male,32.0,0,0,STON/O 2. 3101293,7.9250,NaN,S
704,705,3,"Hansen, Mr. Henrik Juul",male,26.0,1,0,350025,7.8542,NaN,S
813,814,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,NaN,S


In [81]:
# print the number of unique values of each categorical column
for col in X_train.columns:
    if X_train[col].dtype == 'object':
        print(col, ': ', X_train[col].nunique())

Name :  712
Sex :  2
Ticket :  558
Cabin :  117
Embarked :  3


### Preprocessing Piple
We will use a combination of Pipelines and column transformers to setup the data
1. Tranform "Cabin" feature to a binary "has_cabin" feature
2. Setup a column transformer for numerical values which impute with median and StandardScaler
3. Setup a column transformer for categorical features which ignore unknown(s)
4. Setup a column transformer to drop unwanted columns

In [82]:
from sklearn.pipeline import Pipeline
preprocess_pipeline = Pipeline(steps=[])

1. **Tranform "Cabin" feature to a binary "has_cabin" feature**

In [83]:
from sklearn.base import BaseEstimator, TransformerMixin

class DummyTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, cols = None):
        """
        Dummy class to minimize code duplication
        from: https://towardsdatascience.com/customizing-sklearn-pipelines-transformermixin-a54341d8d624
        """
        self.cols = cols
        self.dummies = None
        return None
    
    def fit(self, X = None, y = None):
        return self

    def transform(self, X = None, y = None):
        return self
    

In [84]:
class ImputeCabinTransformer(DummyTransformer):
    """Impute Cabin feature to has_cabin feature as binary"""
    def __init__(self, feature_name, new_name):
        self.feature_name = feature_name
        self.new_name = new_name

    def fit(self, X, y=None):
        X_ = X.rename(columns={self.feature_name:self.new_name})
        self.columns = [col for col in X_.columns]
        return self

    def transform(self, X):
        X_ = X.drop(labels = self.feature_name, axis = 1)
        X_[self.new_name] = X[self.feature_name].notna().replace({True: 1, False: 0})

        return X_


In [85]:
CabinImputer = ImputeCabinTransformer(feature_name = 'Cabin', new_name = 'hasCabin')
preprocess_pipeline.steps.append(('CabinImputer', CabinImputer))

In [86]:
display(preprocess_pipeline)

Pipeline(steps=[('CabinImputer',
                 ImputeCabinTransformer(feature_name='Cabin',
                                        new_name='hasCabin'))])

2. **Setup a column transformer for numerical values which impute with median and StandardScaler**

In [87]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [88]:
numerical_cols = ['Age','Fare']
numerical_pipeline = Pipeline(steps=[
    ('MedianImuter', SimpleImputer(strategy='median')),
    ('StandardScaler', StandardScaler())
])

3. **Setup a column transformer for categorical features which ignore unknown(s)**

In [89]:
categorical_cols = ['Pclass','Sex','Embarked']
categorical_vals = [[1,2,3],['male','female'],['C','Q','S']]
categorical_pipeline = Pipeline(steps=[
    ('OHE',OneHotEncoder(handle_unknown='ignore',sparse_output=False,dtype='int16',
                         categories=categorical_vals))
])

In [90]:
drop_cols = ['PassengerId','Name','Ticket']

ColumnTransform = ColumnTransformer(
    transformers=[
        ('numerical',numerical_pipeline,numerical_cols),
        ('categorical',categorical_pipeline,categorical_cols),
        ('drop','drop',drop_cols)
    ], remainder='passthrough'
)

preprocess_pipeline.steps.append(('ColumnTransform',ColumnTransform))

In [91]:
display(preprocess_pipeline.fit(X_train))

Pipeline(steps=[('CabinImputer',
                 ImputeCabinTransformer(feature_name='Cabin',
                                        new_name='hasCabin')),
                ('ColumnTransform',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numerical',
                                                  Pipeline(steps=[('MedianImuter',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('StandardScaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('categorical',
                                                  Pipeline(steps=[('OHE',
                                                                   OneHotEncoder(categories=[[1,
                                                                                              2,
                                                                                              3],
                                                                                             ['male',
                                                                                              'female'],
                                                                                             ['C',
                                                                                              'Q',
                                                                                              'S']],
                                                                                 dtype='int16',
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Pclass', 'Sex',
                                                   'Embarked']),
                                                 ('drop', 'drop',
                                                  ['PassengerId', 'Name',
                                                   'Ticket'])]))])

In [92]:
print(preprocess_pipeline.fit_transform(X_train))

[[ 1.25364106 -0.07868358  1.         ...  0.          0.
   1.        ]
 [-0.47728355 -0.37714494  0.         ...  0.          0.
   0.        ]
 [ 0.21508629 -0.47486697  0.         ...  0.          0.
   0.        ]
 ...
 [ 0.90745614 -0.35580399  0.         ...  2.          0.
   0.        ]
 [-1.1696534   1.68320121  1.         ...  1.          2.
   1.        ]
 [-0.63114352  0.86074761  1.         ...  0.          1.
   1.        ]]


In [93]:
preprocess_pipeline[-1].get_feature_names_out()

array(['numerical__Age', 'numerical__Fare', 'categorical__Pclass_1',
       'categorical__Pclass_2', 'categorical__Pclass_3',
       'categorical__Sex_male', 'categorical__Sex_female',
       'categorical__Embarked_C', 'categorical__Embarked_Q',
       'categorical__Embarked_S', 'remainder__SibSp', 'remainder__Parch',
       'remainder__hasCabin'], dtype=object)

In [94]:
display(pd.DataFrame(preprocess_pipeline.fit_transform(X_train),columns=preprocess_pipeline[-1].get_feature_names_out()))

,numerical__Age,numerical__Fare,categorical__Pclass_1,categorical__Pclass_2,categorical__Pclass_3,categorical__Sex_male,categorical__Sex_female,categorical__Embarked_C,categorical__Embarked_Q,categorical__Embarked_S,remainder__SibSp,remainder__Parch,remainder__hasCabin
0,1.253641,-0.078684,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,-0.477284,-0.377145,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.215086,-0.474867,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-0.246494,-0.476230,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,-1.785093,-0.025249,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,4.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,-0.631144,-0.480162,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
708,-0.092634,-0.030545,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
709,0.907456,-0.355804,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0
710,-1.169653,1.683201,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,1.0
